In [ ]:
import pandas as pd
import numpy as np
import pymetis as pm

def load_metis_data(path: str):
    df = pd.read_csv(path)
    
    item1_col = df.columns[0]
    item2_col = df.columns[1]
    feature_cols = df.columns[2:]
    
    feature_col = df[feature_cols].sum(axis=1)

    unique_items = pd.concat([df[item1_col], df[item2_col]]).unique()
    item_map = {item: idx for idx, item in enumerate(sorted(unique_items))}
    rev_item_map = {idx: item for idx, item in enumerate(sorted(unique_items))}
    item_count = len(item_map)
    
    adjacency_lists = {i: {} for i in range(item_count)}
    
    for idx, row in df.iterrows():
        weight = int(feature_col.iloc[idx])
        if weight <= 0:
            continue
        
        v1 = item_map[row[item1_col]]
        v2 = item_map[row[item2_col]]
        
        adjacency_lists[v1][v2] = weight
        adjacency_lists[v2][v1] = weight
    
    xadj = [0]
    adjncy = []
    eweights = []
    
    for i in range(item_count):
        neighbors = sorted(adjacency_lists[i].keys())
        for neighbor in neighbors:
            adjncy.append(neighbor)
            eweights.append(adjacency_lists[i][neighbor])
        xadj.append(len(adjncy))
    
    csr_adj = pm.CSRAdjacency(
        np.array(xadj, dtype=np.int32),
        np.array(adjncy, dtype=np.int32)
    )
    eweights = np.array(eweights, dtype=np.int32)
    
    return csr_adj, eweights, rev_item_map

path = '/home/sam/mc-clustering/packages/mc-datagen/dist/1.21.10/matrix.csv'
csr_adj, eweights, vertex_to_item = load_metis_data(path)

In [29]:
from src.util import zip_items

def train_metis(n, adj: pm.CSRAdjacency, weights, rev_item_map):
    _, membership = pm.part_graph(n, adjacency=adj, eweights=weights)
    items = [rev_item_map[i] for i in range(len(membership))]
    return items, membership

items, labels = train_metis(40, csr_adj, eweights, vertex_to_item)
zip_items(items, labels)

,Items,Group
0,minecraft:acacia_boat,23
1,minecraft:acacia_button,22
2,minecraft:acacia_chest_boat,23
3,minecraft:acacia_door,22
4,minecraft:acacia_fence,21
...,...,...
2887,minecraft:zombie_head,37
2888,minecraft:zombie_horse_spawn_egg,19
2889,minecraft:zombie_spawn_egg,17
2890,minecraft:zombie_villager_spawn_egg,18
